In [32]:
n= 10 #10 period
r00= 0.05
u=1.1
d=0.9
q=.5
face_value=100.0

In [33]:
import numpy as np

In [34]:
def short_rate_tree(n,r00,u,d):
    tree = np.zeros((n+1,n+1))
    for i in range(n+1):
        for j in range(i+1):
            tree[i][j]=r00*u**j*d**(i-j)
    return tree

In [35]:
short_rate_tree(n,r00,u,d)

array([[0.05      , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.045     , 0.055     , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.0405    , 0.0495    , 0.0605    , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.03645   , 0.04455   , 0.05445   , 0.06655   , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.032805  , 0.040095  , 0.049005  , 0.059895  , 0.073205  ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.0295245 , 0.0360855 , 0.0441045 , 0.0539055 , 0.0658845 ,
        0.0805255 , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.02657205, 0.03247695, 0.03969405, 0.04851495, 0.

In [36]:
def zero_coupon_bond_price(n,r00,u,d,q,face_value):
    price_tree = np.ones((n+1,n+1))*face_value
    rate_tree = short_rate_tree(n,r00,u,d)
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            price_tree[i][j]=1/(1+rate_tree[i][j])*(q*price_tree[i+1][j+1]+(1-q)*price_tree[i+1][j])
    return price_tree

In [37]:
zero_coupon_bond_price(n,r00,u,d,q,face_value)

array([[ 61.62195812, 100.        , 100.        , 100.        ,
        100.        , 100.        , 100.        , 100.        ,
        100.        , 100.        , 100.        ],
       [ 67.44102962,  61.96508242, 100.        , 100.        ,
        100.        , 100.        , 100.        , 100.        ,
        100.        , 100.        , 100.        ],
       [ 72.8818303 ,  68.06992161,  62.6764023 , 100.        ,
        100.        , 100.        , 100.        , 100.        ,
        100.        , 100.        , 100.        ],
       [ 77.88686151,  73.78022735,  69.09853811,  63.83811117,
        100.        , 100.        , 100.        , 100.        ,
        100.        , 100.        , 100.        ],
       [ 82.42221636,  79.02945886,  75.10481409,  70.61709293,
         65.55598201, 100.        , 100.        , 100.        ,
        100.        , 100.        , 100.        ],
       [ 86.47456207,  83.77759226,  80.61869778,  76.95195323,
         72.7414542 ,  67.96856115, 100. 

In [38]:
zero_coupon_bond_price(4,r00,u,d,q,1)[0][0]

0.8228895735604677

In [39]:
#Compute the price of a forward contract on the same ZCB of the previous question where the forward contract matures at time t=4
def forward_price(n,r00,u,d,q,face_value,maturity = 4):
    rate_tree = short_rate_tree(n,r00,u,d)
    expect_B = zero_coupon_bond_price(4,r00,u,d,q,1)[0][0]
    tree = zero_coupon_bond_price(n,r00,u,d,q,face_value)[0:maturity+1,0:maturity+1]
    for i in range(maturity-1,-1,-1):
        for j in range(i+1):
            tree[i][j]=1/(1+rate_tree[i][j])*(q*tree[i+1][j+1]+(1-q)*tree[i+1][j])
    return tree[0][0]/expect_B

In [40]:
forward_price(n,r00,u,d,q,face_value,maturity = 4)

74.88484493844841

In [41]:
def future_price(n,r00,u,d,q,face_value,maturity = 4):
    tree = zero_coupon_bond_price(n,r00,u,d,q,face_value)[0:maturity+1,0:maturity+1]
    for i in range(maturity-1,-1,-1):
        for j in range(i+1):
            tree[i][j]=(q*tree[i+1][j+1]+(1-q)*tree[i+1][j])
    return tree[0][0]

In [42]:
future_price(n,r00,u,d,q,face_value,maturity = 4)

74.82458063139569

In [66]:
def american_call_option_on_zcb(n,r00,u,d,q,face_value,expiration = 6, K=80.0):
    tree = zero_coupon_bond_price(n,r00,u,d,q,face_value)[0:expiration+1,0:expiration+1]
    f=np.vectorize(lambda x:max(float(x)-float(K),0.0))
    payoff = f(tree)
    rate_tree = short_rate_tree(n,r00,u,d)
    for i in range(expiration-1,-1,-1):
        for j in range(i+1):
            payoff[i][j]=max(payoff[i][j],1/(1+rate_tree[i][j])*(q*payoff[i+1][j+1]+(1-q)*payoff[i+1][j]))
    return payoff

In [67]:
american_call_option_on_zcb(n,r00,u,d,q,face_value,expiration = 6, K=80.0)

array([[ 2.35721516, 20.        , 20.        , 20.        , 20.        ,
        20.        , 20.        ],
       [ 3.39350009,  1.55665175, 20.        , 20.        , 20.        ,
        20.        , 20.        ],
       [ 4.64733524,  2.44507995,  0.83945526, 20.        , 20.        ,
        20.        , 20.        ],
       [ 6.03029794,  3.64080669,  1.49141612,  0.28906847, 20.        ,
        20.        , 20.        ],
       [ 7.42283086,  5.07737375,  2.5286355 ,  0.61661196,  0.        ,
        20.        , 20.        ],
       [ 8.76878625,  6.5638874 ,  3.9980147 ,  1.30708787,  0.        ,
         0.        , 20.        ],
       [10.04745952,  8.00790104,  5.59359608,  2.75509419,  0.        ,
         0.        ,  0.        ]])

In [80]:
def forward_equation_lattice(n,r00,u,d,q=.5):
    tree = np.zeros((n+1,n+1))
    tree[0][0]=1
    rate_tree= short_rate_tree(n,r00,u,d)
    for i in range(1,n+1):
        tree[i][0]=1/(1+rate_tree[i-1][0])*(1-q)*tree[i-1][0]
        tree[i][i]=1/(1+rate_tree[i-1][i-1])*q*tree[i-1][i-1]
    for i in range(2,n+1):
        for j in range(1,i):
            tree[i,j]=(1-q)*1/(1+rate_tree[i-1,j])*tree[i-1,j]+q*1/(1+rate_tree[i-1,j-1])*tree[i-1,j-1]
    return tree

In [81]:
forward_equation_lattice(n,r00,u,d,q=.5)

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.76190476e-01, 4.76190476e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.27842333e-01, 4.53525023e-01, 2.25682690e-01, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.09486945e-01, 3.25554131e-01, 3.22471095e-01, 1.06403909e-01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.28182475e-02, 2.08652880e-01, 3.08744251e-01, 2.02791907e-01,
        4.98822880e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+

#Compute the initial value of a forward-starting swap that begins at t=1 with maturity t=10 and a fixed rate of 4.5%.

In [78]:
def forward_starting_swap_price(n,r00,u,d,q=.5,notional_principal = 1000000.0,fixed_rate = 0.045):
    lattice = forward_equation_lattice(n,r00,u,d,q=.5)
    rate_tree= short_rate_tree(n,r00,u,d)
    s = 0 
    for i in range(1,n+1):
        for j in range(i+1):
            s += -(fixed_rate - rate_tree[i][j])/(1+rate_tree[i][j])*lattice[i,j]
    return s*notional_principal

In [76]:
forward_starting_swap_price(n,r00,u,d,q=.5,notional_principal = 1000000.0,fixed_rate = 0.045)

33374.242062163794

In [77]:
0.1331/1.1831

0.11250105654636125

In [83]:
def swap_price_lattice(n,r00,u,d,q=.5,notional_principal = 1000000.0,fixed_rate = 0.045):
    rate_tree= short_rate_tree(n,r00,u,d)
    payoff = np.zeros((n+1,n+1))
    for i in range(n+1):
        payoff[n,i]=(rate_tree[n,i]-fixed_rate)/(1+rate_tree[n,i])
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            payoff[i,j]= (rate_tree[i,j]-fixed_rate+q*payoff[i+1,j+1]+(1-q)*payoff[i+1,j])/(1+rate_tree[i,j])
    return payoff

In [85]:
swap_price_lattice(n,r00,u,d,q=.5,fixed_rate = 0.045)

array([[ 0.03813615,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.00226851,  0.07235442,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.03483091,  0.03008972,  0.1025781 ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.05929641, -0.00418671,  0.05834504,  0.12822312,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.07568293, -0.0301326 ,  0.02228614,  0.0818577 ,  0.14855504,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.08423175, -0.04770966, -0.00516187,  0.04390843,  0.09982271,
         0.16262731,  0.        ,  0.        ,  0.        ,  0.        ,
         0.   

In [90]:
def swaption_price(n,r00,u,d,q=.5,notional_principal = 1000000.0,fixed_rate = 0.045,t=5,K=0.0):
    lattice = swap_price_lattice(n,r00,u,d,q=.5,fixed_rate = 0.045)
    rate_tree= short_rate_tree(n,r00,u,d)
    f= np.vectorize(lambda x: max(x-K,0.0))
    payoff = f(lattice)[0:t+1,0:t+1]
    for i in range(t-1,-1,-1):
        for j in range(i+1):
            payoff[i,j]=(q*payoff[i+1,j+1]+(1-q)*payoff[i+1,j])/(1+rate_tree[i,j])
    return payoff[0,0]*notional_principal

In [91]:
swaption_price(n,r00,u,d,q=.5,notional_principal = 1000000.0,fixed_rate = 0.045,t=5,K=0.0)

26311.07949019227